### Install Library

In [1]:
pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6  datasets==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)


In [2]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
import huggingface_hub
huggingface_hub.login()

# 1. set model and dataset

★ set point:  set your own model path in huggingface★

In [5]:
# set base model path
base_model = "hyokwan/llama3-ko-8b-hkcode"
# set new model path
new_model = "Llama3-hkcode-Ko-8b-youtube-meta"

In [6]:
# new dataset
dataset_namehk = "hyokwan/llama3data_hkcodenew"
datasethk = load_dataset(dataset_namehk, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/20 [00:00<?, ? examples/s]

In [7]:
def create_text_column(example):
    # 'text' 컬럼 생성
    text = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    example["text"] = text
    return example

# 'text' 컬럼 생성
datasethk = datasethk.map(create_text_column)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [8]:
datasethk[2]

{'instruction': '하이테크과정 원서접수는 언제부터 하나요?',
 'output': '   ▶ 접수기간: 매년 11월 초부터 다음해 2월까지\n   ▶ 접수방법: 인터넷 및 방문 접수',
 'input': '',
 'text': '### Instruction:\n하이테크과정 원서접수는 언제부터 하나요?\n\n### Response:\n   ▶ 접수기간: 매년 11월 초부터 다음해 2월까지\n   ▶ 접수방법: 인터넷 및 방문 접수'}

In [9]:
datasethk

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 20
})

# 2. Config efficient fine-tuning with low-rank adaptation.

In [10]:
# 현재 사용 중인 GPU의 주요 아키텍처 버전을 반환 8버전 이상 시 bfloat16 활용
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# BitsAndBytesConfig 객체활용 양자화 설정
# 모델을 4비트 양자화하여 로드
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.8 MB/s eta 0:00:00


# 3. Load Pre-trained Language Model

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
    # device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

# 4. Load Pre-trained Language Model Tokenizer

In [12]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# Must add EOS_TOKEN at response last line
tokenizer.pad_token = tokenizer.eos_token
# ★수정 포인트!!! 기존 # tokenizer.padding_side = "right"
EOS_TOKEN = tokenizer.eos_token
def prompt_eos(sample):
    sample['text'] = sample['text']+EOS_TOKEN
    return sample
datasethk = datasethk.map(prompt_eos)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [13]:
datasethk[1]

{'instruction': '하이테크과정 학사일정은 어떻게 되나요?',
 'output': '   ㅁ 학사일정은 다음과 같습니다.\n     [ 주간1년과정 ]\n    ㅇ 입학 및 수료 : 매년 3월 입학, 다음해 2월 수료\n    ㅇ 수업시간 : 매주 월요일~금요일, 09:00 ~ 17:40(50분 수업 후 10분 휴식_일부 요일은 6교시)\n    ㅇ 방학 : 하계방학 7~8월 중 4주 이상, 동계방학 12~1월 중 4주 이상\n    ※ 매년 조기취업 일부가능, 매년 11월 초순부터 현장실습(취업) 가능',
 'input': '',
 'text': '### Instruction:\n하이테크과정 학사일정은 어떻게 되나요?\n\n### Response:\n   ㅁ 학사일정은 다음과 같습니다.\n     [ 주간1년과정 ]\n    ㅇ 입학 및 수료 : 매년 3월 입학, 다음해 2월 수료\n    ㅇ 수업시간 : 매주 월요일~금요일, 09:00 ~ 17:40(50분 수업 후 10분 휴식_일부 요일은 6교시)\n    ㅇ 방학 : 하계방학 7~8월 중 4주 이상, 동계방학 12~1월 중 4주 이상\n    ※ 매년 조기취업 일부가능, 매년 11월 초순부터 현장실습(취업) 가능<|eot_id|>'}

In [15]:
 def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    # generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=128009)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")
key="HKCODE 유튜브는 누가 운영하나요?"
prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

'<|begin_of_text|> 한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.<|end_of_text|>'

# 5. Config training parameter for LoRA (Parameter-Efficient Fine-Tuning (PEFT)

https://huggingface.co/docs/peft/conceptual_guides/lora

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# 6. Train Model

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=datasethk,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Step,Training Loss
25,2.122600
50,0.395000
75,0.041100
100,0.030200


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=100, training_loss=0.6471986657381058, metrics={'train_runtime': 275.805, 'train_samples_per_second': 1.45, 'train_steps_per_second': 0.363, 'total_flos': 2804685392609280.0, 'train_loss': 0.6471986657381058, 'epoch': 20.0})

# 7. Verify

In [ ]:
 def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    # generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=128009)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")
key="스마트금융과는 어디에 위치하나요?"
prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

'<|begin_of_text|> 스마트금융과는 서울강서구 열리经济zone 천호동 406-1에서 운영됩니다.<|end_of_text|>'

In [ ]:
key="스마트금융과는 무엇을 가르치나요?"
prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

'<|begin_of_text|> 스마트금융과는 사이버보안 및 인공지능 분야를 가르치고 있습니다. 세계 첫 번째 스마트금융과를 2013년 서울강서캠퍼스에-establishment(설립)되었습니다. 서울강서캠퍼스에 있는 모든 캠퍼스는 스마트금융과가 establish(establishment)된 �서스[서면]가 있습니다. 예를 들어 서울강서�ーパー스에 있는 스마트금융과는 2013년 establish(establishment)된 스마트금융과입니다.<|end_of_text|>'

In [ ]:
key="하이테크과정은 어떤 교육과정인가요?"
prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

'<|begin_of_text|> 하이테크과정은 4차 산업혁명 선도 및 유망·성장동력 분야 인력양성을 위한 취업 역량 향상 과정입니다. 서울강서캠퍼스의 경우 하이테크과정에는 스마트금융과, 사이버보안과, 출판편집디자인과가 있습니다.<|end_of_text|>'

In [ ]:
savePath = "/content/gdrive/MyDrive/Colab Notebooks/llama/02. Fine Tuning/llama3_mete_retrain0602"
trainer.save(savePath)